In [61]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import copy

df_hospital=pd.read_csv(r'../data/hospital_resources/Hospital_resources_hanmin.csv', encoding='ISO-8859-1')
df_hospital.head()

fig = go.Figure()
fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = df_hospital['Longitude'],
    lat = df_hospital['Latitude'],
    hoverinfo = 'text',
    text = df_hospital['Inst'],
    mode = 'markers',
    marker = dict(
        size = 5,
        color = 'rgb(0, 0, 255)',
        line = dict(
            width = 3,
            color = 'rgba(68, 68, 68, 0)'
        ))
    ))

fig.update_layout(
    title_text = 'Resource reallocation',
    showlegend = False,
    geo = dict(
        resolution = 50, 
        scope = 'europe',
#         center = {'lat': (data.lat.min() + data.lat.max())/2,
#                 'lon': (data.lon.min() + data.lon.max())/2},
        projection_type = 'azimuthal equal area',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
)

In [36]:
df_hospital

,ID,KT,Inst,Adr,PersA,PersP,BettenStatA,pBettenBelStatA,ICU_beds,Ventilators,Latitude,Longitude,Unnamed: 12
0,0,BS,UniversitÃ_tsspital Basel,Hebelstrasse 32,913.50,1595.22,633.000000,92.463806,38,28,47.561196,7.581541,NaN
1,1,GE,Les HÃÂ«pitaux Universitaires de GenÃÂ¬ve HUG,Rue GabrielleÃ¥_PerretÃ¥_Gentil 4,1829.32,4498.96,1054.282192,85.289218,63,47,46.193711,6.148856,NaN
2,2,VD,CHUV Centre Hospitalier Universitaire Vaudois,Rue du Bugnon 21,1653.73,3400.37,929.000000,82.588437,56,41,46.524749,6.639659,NaN
3,3,ZH,UniversitÃ_tsspital ZÃ_rich,RÃ_mistrasse 100,1221.75,2574.21,953.884931,78.680694,57,42,47.376736,8.549199,NaN
4,4,BE,Insel Gruppe AG (universitÃ_r),Freiburgstrasse 18,1355.66,2739.25,876.298630,83.329008,53,39,46.947455,7.424801,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,230,GE,Clinique de Carouge,Avenue CardinalÃ¥_Mermillod 1,14.78,40.46,15.000000,28.566210,1,1,46.186931,6.142615,NaN
63,257,ZH,UniversitÃ_ts-Kinderspital ZÃ_rich das Spita...,Steinwiesstrasse 75,302.29,683.98,167.128767,75.649979,10,7,47.372031,8.557767,NaN
64,258,SG,Ostschweizer Kinderspital,Claudiusstrasse 6,85.25,272.69,74.000000,81.728989,4,3,47.431105,9.387304,NaN
65,259,BS,UniversitÃ_ts-Kinderspital beider Basel (UKBB),Spitalstrasse 33,157.55,411.82,118.501370,72.873558,7,5,47.563624,7.581830,NaN


In [35]:
data = {'orig_lat': [-1]*2211, 
        'orig_lon': [-1]*2211, 
        'dest_lat': [-1]*2211,
        'dest_lon': [-1]*2211,
        'orig_hosp': [-1]*2211,
        'dest_hosp': [-1]*2211,
        'num_ven_tansp': [-1]*2211} 
df_ven_reloc = pd.DataFrame(data)
df_ven_reloc['orig_lat'][0] = 100
type(df_ven_reloc['orig_lat'])
print(df_ven_reloc)


      orig_lat  orig_lon  dest_lat  dest_lon  orig_hosp  dest_hosp  \
0          100        -1        -1        -1         -1         -1   
1           -1        -1        -1        -1         -1         -1   
2           -1        -1        -1        -1         -1         -1   
3           -1        -1        -1        -1         -1         -1   
4           -1        -1        -1        -1         -1         -1   
...        ...       ...       ...       ...        ...        ...   
2206        -1        -1        -1        -1         -1         -1   
2207        -1        -1        -1        -1         -1         -1   
2208        -1        -1        -1        -1         -1         -1   
2209        -1        -1        -1        -1         -1         -1   
2210        -1        -1        -1        -1         -1         -1   

      num_ven_tansp  
0                -1  
1                -1  
2                -1  
3                -1  
4                -1  
...             ...  
2206 

In [49]:
ven_aloc_solu = np.load(r'../data/hospital_resources/Transported_ventilators.npy', allow_pickle=True)


0

In [58]:
# loop df_hospital to create line every
ind = 0
for orig_ind in range(len(df_hospital)-1):
    for dest_ind in range(orig_ind+1, 67):
#         print(df_hospital['Latitude'][orig_ind])
        df_ven_reloc['orig_lat'][ind] = df_hospital['Latitude'][orig_ind]
        df_ven_reloc['orig_lon'][ind] = df_hospital['Longitude'][orig_ind] 
        df_ven_reloc['orig_hosp'][ind] = df_hospital['Inst'][orig_ind]  # from hospital
        
        df_ven_reloc['dest_lat'][ind] = df_hospital['Latitude'][dest_ind] 
        df_ven_reloc['dest_lon'][ind] = df_hospital['Longitude'][dest_ind]  
        df_ven_reloc['dest_hosp'][ind] = df_hospital['Inst'][dest_ind]  # to hospital
        df_ven_reloc['num_ven_tansp'][ind] = ven_aloc_solu[orig_ind, dest_ind]  
#         print(ven_aloc_solu[orig_ind, dest_ind] )
        ind += 1
# print(df_ven_reloc)    

C:\Users\caha\PycharmProjects\ch_vs_virus_416\venv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\caha\PycharmProjects\ch_vs_virus_416\venv\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\caha\PycharmProjects\ch_vs_virus_416\venv\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\caha\Pycha

In [59]:
df_ven_reloc.to_csv('../data/hospital_resources/df_ven_reloc.csv')

In [62]:
for i in range(len(df_ven_reloc)):
    fig.add_trace(
        go.Scattergeo(
            locationmode = 'USA-states',
            lon = [df_ven_reloc['orig_lon'][i], df_ven_reloc['dest_lon'][i]],
            lat = [df_ven_reloc['orig_lat'][i], df_ven_reloc['dest_lat'][i]],
            mode = 'lines',
            line = dict(width = 1,color = 'red'),
            opacity = float(df_ven_reloc['num_ven_tansp'][i]) / float(df_ven_reloc['num_ven_tansp'].max()),
        )
    )

fig.update_layout(
    title_text = 'Ventilator reallocation results',
    showlegend = False,
    geo = dict(
        resolution = 50, 
        scope = 'europe',
#         center = {'lat': (data.lat.min() + data.lat.max())/2,
#                 'lon': (data.lon.min() + data.lon.max())/2},
        projection_type = 'azimuthal equal area',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
)

fig.show()